In [1]:
#Set up
from arcgis import GIS
from arcgis import GeoAccessor
gis = GIS('home')
#gis = GIS("https://dukeuniv.maps.arcgis.com",client_id='YmtqqKYoHULH1M2l')
#print("Successfully logged in as: " + gis.properties.user.username)

In [5]:
gis.content.search("GEBCO 2021",outside_org=True)

[<Item title:"National Geographic Map" type:Web Map owner:esri>,
 <Item title:"National Geographic World Map" type:Map Image Layer owner:esri>,
 <Item title:"Elevation Coverage Map" type:Map Image Layer owner:esri>,
 <Item title:"GEBCO2021 Data Coverage" type:Code Attachment owner:hyaroperez_unh>,
 <Item title:"CommunitySurvey_DataLocation" type:Feature Layer Collection owner:roperezja_NIWA>,
 <Item title:"GEBCO_basemap_NCEI" type:Map Image Layer owner:nicolas@ouutu.com>,
 <Item title:"The Nippon Foundation - GEBCO Seabed 2030 Project - South and West Pacific Regional Mapping Community Meeting_fieldworker" type:Feature Layer Collection owner:roperezja_NIWA>,
 <Item title:"Make_Transits_Matter" type:Web Mapping Application owner:hyaroperez_unh>,
 <Item title:"fviypfm_kyfcefnqwvkz" type:Service Definition owner:migratorybirds_audubon>,
 <Item title:"fviypfm_gpsdhflxcdvy" type:Vector Tile Layer owner:migratorybirds_audubon>]

In [ ]:
#Search for Bathymetry data
search_results = gis.content.search("TopoBathy",outside_org=True)
search_results

In [ ]:
#Get the item
the_item = search_results[0]
the_item

In [ ]:
#Get the one and only layer
bathy_layer = the_item.layers[0]

In [ ]:
#View the layer
m = gis.map("Cape Cod")
m.add_layer(bathy_layer)
#m

In [ ]:
#Get the spatial reference
bathy_sr = the_layer.properties['spatialReference']
bathy_sr

In [ ]:
#Read in Eliza's shapefile
filename = 'U:\\ec308\\FinalProject\\Data\\BOEMWindLeaseOutlines_6_30_2022.shp'
sdf_leases = GeoAccessor.from_featureclass(filename)
sdf_leases.head(2)

In [ ]:
#Show the spatial reference
sdf_leases.spatial.sr

In [ ]:
#Project the data
sdf_leases.spatial.project(bathy_sr)

In [ ]:
#Confirm the reprojection
sdf_leases.spatial.sr

In [ ]:
#Convert to layer
#lyr_leases = sdf_leases.spatial.to_featurelayer(
#    title='temp_BOEM_Leases',
#)
lyr_leases = gis.content.get('e169db88cb54446da2baf1d6d2ec9388')
lyr_leases

### Raster analysis

In [ ]:
#Import the function
from arcgis.raster.analytics import *

In [ ]:
#Sample
result = sample(
    input_rasters=bathy_layer,
    input_location_data=lyr_leases,
    resampling_type='Nearest',
    unique_id_field='FID',
    process_as_multidimensional=True
)

In [ ]:
#Apply the summarize_raster_within function
result = summarize_raster_within(
    input_zone_layer=lyr_leases,
    input_raster_layer_to_summarize=bathy_layer,
    zone_field='PROTRACT_1'
)

In [ ]:
lyr_leases

In [ ]:
#Apply the zonal_statistics_as_table function
result = zonal_statistics_as_table(
    input_zone_raster_or_features=lyr_leases,
    input_value_raster=bathy_layer,
    zone_field='FID',
    statistic_type='ALL',
    gis=gis,
    output_name='tempRasterAnalysis'
)

In [ ]:
result.status